In [ ]:
# !pip install -e ".[icechunk]"
# !pip install git+https://github.com/mpiannucci/kerchunk@v3
# !pip install fsspec s3fs

In [1]:
!pip show icechunk

Name: icechunk
Version: 0.1.0a7
Summary: Transactional storage engine for Zarr designed for use on cloud object storage
Home-page: https://github.com/earth-mover/icechunk
Author: Earthmover PBC
Author-email: 
License: Apache-2.0
Location: /Users/aimeebarciauskas/github/virtualizarr/venv/lib/python3.12/site-packages
Requires: zarr
Required-by: 


In [2]:
import warnings

import fsspec
import xarray as xr
from icechunk import IcechunkStore, StorageConfig, StoreConfig, VirtualRefConfig

from virtualizarr import open_virtual_dataset

warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
fs = fsspec.filesystem("s3", anon=True)

oisst_files = fs.glob(
    "s3://noaa-cdr-sea-surface-temp-optimum-interpolation-pds/data/v2.1/avhrr/202408/oisst-avhrr-v02r01.*.nc"
)

oisst_files = sorted(["s3://" + f for f in oisst_files])

In [4]:
so = dict(anon=True, default_fill_cache=False, default_cache_type="none")

virtual_datasets = [
    open_virtual_dataset(url, indexes={}, reader_options={"storage_options": so})
    for url in oisst_files[0:2]
]

In [5]:
virtual_ds = xr.concat(
    virtual_datasets,
    dim="time",
    coords="minimal",
    compat="override",
    combine_attrs="override",
)

In [18]:
!rm -rf ./noaa-cdr-icechunk/

In [19]:
storage_config = StorageConfig.filesystem("./noaa-cdr-icechunk")
virtual_ref_store_config = StoreConfig(
    virtual_ref_config=VirtualRefConfig.s3_anonymous(region="us-east-1"),
)

In [20]:
store = await IcechunkStore.create(
    storage=storage_config, config=virtual_ref_store_config, read_only=False
)

In [21]:
virtual_ds.virtualize.to_icechunk(store)

/Users/aimeebarciauskas/github/virtualizarr/virtualizarr/writers/icechunk.py:272: RuntimeWarning: coroutine 'IcechunkStore.set_virtual_ref' was never awaited
  store.set_virtual_ref(
/Users/aimeebarciauskas/github/virtualizarr/virtualizarr/writers/icechunk.py:272: RuntimeWarning: coroutine 'IcechunkStore.set_virtual_ref' was never awaited
  store.set_virtual_ref(
/Users/aimeebarciauskas/github/virtualizarr/virtualizarr/writers/icechunk.py:272: RuntimeWarning: coroutine 'IcechunkStore.set_virtual_ref' was never awaited
  store.set_virtual_ref(
/Users/aimeebarciauskas/github/virtualizarr/virtualizarr/writers/icechunk.py:272: RuntimeWarning: coroutine 'IcechunkStore.set_virtual_ref' was never awaited
  store.set_virtual_ref(
/Users/aimeebarciauskas/github/virtualizarr/virtualizarr/writers/icechunk.py:272: RuntimeWarning: coroutine 'IcechunkStore.set_virtual_ref' was never awaited
  store.set_virtual_ref(
/Users/aimeebarciauskas/github/virtualizarr/virtualizarr/writers/icechunk.py:272: Run

In [16]:
await store.commit("first 2 days of 202408 data")

'PZMXKYCPJQRRFXV33Q7G'

In [17]:
await store._store.get("time/c/0")

KeyNotFound: time/c/0

In [ ]:
ds = xr.open_zarr(store, consolidated=False, zarr_format=3)
ds

## Append

In [ ]:
virtual_datasets_a = [
    open_virtual_dataset(
        url, indexes={}, reader_options={"storage_options": {"anon": True}}
    )
    for url in oisst_files[2:4]
]

In [ ]:
virtual_ds_a = xr.concat(
    virtual_datasets_a,
    dim="time",
    coords="minimal",
    compat="override",
    combine_attrs="override",
)

In [ ]:
virtual_ds_a

In [ ]:
append_store = IcechunkStore.open_existing(
    storage=storage_config, config=virtual_ref_store_config, read_only=False
)

In [ ]:
await append_store._store.get("time/c/0")

In [ ]:
virtual_ds_a.virtualize.to_icechunk(append_store, append_dim="time")

In [ ]:
append_store.commit("wrote 2 more days of data")

In [ ]:
read_store = IcechunkStore.open_existing(
    storage=storage_config, config=virtual_ref_store_config, read_only=True
)

In [ ]:
await read_store._store.get("time/c/0")

In [ ]:
ds = xr.open_zarr(read_store, consolidated=False, zarr_format=3)
ds